# PPG Field Study Dataset
## 0/ Imports des modules

In [8]:
import pickle
import pandas as pd

## 1/ Chargement des données

In [9]:
path = "./data/S1/S1.pkl"

In [10]:
with open(path, "rb") as f:
    data = pickle.load(f, encoding="latin-1")

## 2/ Analyse des données

In [11]:
data.keys()

dict_keys(['rpeaks', 'signal', 'label', 'activity', 'questionnaire', 'subject'])

In [12]:
pd.DataFrame(data["signal"])

,chest,wrist
ACC,"[[0.8564000129699707, -0.06779998540878296, -0...","[[-0.765625, -0.078125, 0.671875], [-0.765625,..."
ECG,"[[0.015609741210937498], [0.01812744140625], [...",NaN
EMG,"[[-1.5], [-1.5], [-1.5], [-1.5], [-1.5], [-1.5...",NaN
EDA,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[4.722437], [4.728843], [4.718594], [4.717312..."
Temp,"[[-273.15], [-273.15], [-273.15], [-273.15], [...",NaN
Resp,"[[4.44183349609375], [4.45098876953125], [4.47...",NaN
BVP,NaN,"[[7.28], [6.33], [5.46], [4.6], [3.74], [2.86]..."
TEMP,NaN,"[[32.13], [32.16], [32.16], [32.16], [32.16], ..."


## 3/ Création de DataFrame

In [83]:
dfs  = []
pbms = []
oks  = []
for key in data.keys():
    try:
        df = pd.DataFrame(data[key])
        df.dropna()
        dfs.append(df)
        oks.append(key)
    except Exception as e:
        pbms.append(key + str(e))
print(f"clefs ok = {oks}")
print(f"clefs avec un pbms = {pbms}")

clefs ok = ['rpeaks', 'signal', 'label', 'activity']
clefs avec un pbms = ['questionnaireIf using all scalar values, you must pass an index', 'subjectDataFrame constructor not properly called!']


### 3.1/ Regarder les DataFrames

In [85]:
for df in dfs:
    print(df.sample(8))

0
9547  5361483
5246  2887918
5678  3112682
1215   771021
7393  4139420
2874  1568289
766    596892
1965  1161409
                                                  chest  \
ECG   [[0.015609741210937498], [0.01812744140625], [...   
EDA   [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....   
Resp  [[4.44183349609375], [4.45098876953125], [4.47...   
ACC   [[0.8564000129699707, -0.06779998540878296, -0...   
BVP                                                 NaN   
Temp  [[-273.15], [-273.15], [-273.15], [-273.15], [...   
TEMP                                                NaN   
EMG   [[-1.5], [-1.5], [-1.5], [-1.5], [-1.5], [-1.5...   

                                                  wrist  
ECG                                                 NaN  
EDA   [[4.722437], [4.728843], [4.718594], [4.717312...  
Resp                                                NaN  
ACC   [[-0.765625, -0.078125, 0.671875], [-0.765625,...  
BVP   [[7.28], [6.33], [5.46], [4.6], [3.74], [2.86]...  
Temp  

In [74]:
df = dfs[0]
df.plot()
df = dfs[1].sample(1)
df.plot()
df = dfs[2]
df.plot()
df = dfs[3]
df.plot()

TypeError: no numeric data to plot